## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-16-09-40-02 +0000,nypost,"‘X-Men’ star Shawn Ashmore, wife break silence...",https://nypost.com/2025/07/16/us-news/x-men-ac...
1,2025-07-16-09-39-25 +0000,bbc,UK inflation hits highest for almost a year an...,https://www.bbc.com/news/articles/c3en2enpy7po
2,2025-07-16-09-34-12 +0000,bbc,When will interest rates go down again?,https://www.bbc.com/news/articles/c3dky111m40o
3,2025-07-16-09-34-00 +0000,wsj,Indonesia’s Central Bank Cuts Rates to Bolster...,https://www.wsj.com/articles/indonesias-centra...
4,2025-07-16-09-26-18 +0000,bbc,Co-op boss confirms all 6.5m members had data ...,https://www.bbc.com/news/articles/cql0ple066po


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2322/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
168,trump,16
16,inflation,6
176,new,6
40,data,6
328,ukraine,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
50,2025-07-16-00-00-00 +0000,wsj,"President Trump wants tariffs, the higher the ...",https://www.wsj.com/economy/trade/forget-taco-...,37
46,2025-07-16-01-00-00 +0000,wsj,The impact of President Trump’s whirlwind six ...,https://www.wsj.com/economy/trump-effect-start...,35
69,2025-07-15-19-03-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...,34
84,2025-07-15-11-15-55 +0000,nyt,E.U. Nations Plan Retaliatory Tariffs Against ...,https://www.nytimes.com/2025/07/15/world/europ...,32
43,2025-07-16-01-27-41 +0000,nypost,Joe Rogan rips DOJ’s handling of Epstein files...,https://nypost.com/2025/07/15/us-news/joe-roga...,31


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
50,37,2025-07-16-00-00-00 +0000,wsj,"President Trump wants tariffs, the higher the ...",https://www.wsj.com/economy/trade/forget-taco-...
24,28,2025-07-16-06-35-00 +0000,wsj,U.K. Inflation Heats Up as Bank of England Sta...,https://www.wsj.com/economy/u-k-inflation-heat...
48,25,2025-07-16-00-23-39 +0000,bbc,Thousands of Afghans were moved to UK in secre...,https://www.bbc.com/news/articles/cvg8zy78787o
37,21,2025-07-16-02-33-54 +0000,nypost,Beloved teacher dies in 100-foot fall just 2 m...,https://nypost.com/2025/07/15/us-news/beloved-...
26,20,2025-07-16-05-41-46 +0000,nypost,"Burger King worker, mom of 3 Nykia Hamilton fo...",https://nypost.com/2025/07/16/us-news/burger-k...
52,19,2025-07-15-23-23-54 +0000,nypost,"Zohran Mamdani sticks to socialist guns, ‘tax ...",https://nypost.com/2025/07/15/us-news/zohran-m...
0,18,2025-07-16-09-40-02 +0000,nypost,"‘X-Men’ star Shawn Ashmore, wife break silence...",https://nypost.com/2025/07/16/us-news/x-men-ac...
20,18,2025-07-16-07-40-42 +0000,bbc,"Twenty killed in crowd surge at Gaza aid site,...",https://www.bbc.com/news/articles/cg4rwrkdlzxo
69,18,2025-07-15-19-03-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...
35,17,2025-07-16-03-05-26 +0000,nypost,16-year-old stabbed to death inside Bronx deli...,https://nypost.com/2025/07/15/us-news/16-year-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
